In [112]:
import pandas as pd  
import numpy as np
from gekko import GEKKO
from math import e
import warnings
warnings.filterwarnings("ignore")

In [113]:
term_structure = [0.9724765, ###term_structure[0] corresponds to the D(T) value for D(0.5)
0.9445693, ###term_structure[1] corresponds to the D(T) value for D(1.0)
0.9163238,
0.8879337,
0.8597406,
0.8320438,
0.805077,
0.7789041,
0.7534803,
0.7287341,
0.70458,
0.6810105,
0.6580423,
0.6356925,
0.613977,
0.5929047,
0.5724819,
0.5527129,
0.533601,
0.5151482,
0.4973522,
0.480198,
0.4636652,
0.4477312,
0.4323713,
0.4175595,
0.4032682,
0.3894683,
0.3761301,
0.363223]

In [114]:
vol_data = [0.1, ### vol_data[0] corresponds to the vol value for D(1.0)
0.12, ### vol_data[1] corresponds to the vol value for D(1.5)
0.135,
0.15,
0.16,
0.162,
0.164,
0.162,
0.16,
0.157,
0.154,
0.151,
0.148,
0.145,
0.142,
0.139,
0.136,
0.133,
0.13,
0.127,
0.125,
0.123,
0.12,
0.118,
0.116,
0.114,
0.112,
0.11,
0.108]

In [115]:
df = pd.DataFrame()  
df['term_structure'] = term_structure

In [116]:
tree = pd.DataFrame()

In [117]:
### have to initialize dataframe with 31 rows to fit the entire tree
list = [np.NaN]
temp_list = 31*list
tree['0'] = temp_list

In [118]:
### adding in the other columns
temp_list2 = []
for i in range(1, 31):
    temp_list2.append(str(i))
temp_df = pd.DataFrame(columns=temp_list2)
tree = tree.join(temp_df, how="outer")

In [124]:
### D(0.5) = $1 / (1 + r/2), r = 6mt rate today, back out r vvvv
m = GEKKO()
r1 = m.Var()
m.Equation(df['term_structure'][0] == 1 / (1 + r1/2))
m.solve(disp = False)
tree['1'][0] = r1.value[0]
print(r1.value[0]) ### must treat r1.value as a list to get solution into float

0.056603786577


In [125]:
### D(1.0) = (1 / (1 + r/2)) * (0.5 * (1 / (1 + ru/2)) + 0.5 * (1 / (1 + rd/2)))
m = GEKKO()
ru = m.Var()
rd = ru * (e**(-2 * vol_data[0] * 0.70710678))
m.Equation(df['term_structure'][1] == (1 / (1 + r1[0]/2)) * (0.5 * (1 / (1 + ru/2)) + 0.5 * (1 / (1 + rd/2))))
m.solve(disp = False)
print("ru = ", ru.value[0]) ### must treat r1.value as a list to get solution into float
print('rd = ', ru[0] * (e**(-2 * vol_data[0] * 0.70710678)))
tree['2'][0] = ru.value[0]
tree['2'][1] = ru[0] * (e**(-2 * vol_data[0] * 0.70710678))


ru =  0.063269915497
rd =  0.05492609704411437


In [126]:
def avg_r_computer(index):
    """
    avg_r_computer assigns an average equation used in BDT model to a variable
    :param index: the name of the column/period in tree being computed, D(T) = D(index/2)
    """
    
    temp_range = 2**(index - 2)
    coef = (1/2)**(index - 1)
    m = GEKKO()
    r = m.Var() ### this is r*
    r_list = [r] ### this is a list of all the ruuu, ruud, rudd... in terms of r 
    
    temp_index = temp_range - 1
    for i in range(temp_index):
        coef_2 = 2*temp_index + 2
        r_list.append(r * e**(-coef_2 * vol_data[] * 0.70710678))  ### sqrt(0.5) = 0.70710678
    
    for i in range(temp_range): ### this calculates the avg equation most inside, at the index th level
        if(i = 0):
            
            exec("var" + str(i) + " = (coef/(1 + tree[index - 1][i]) + coef/(1 + tree[index - 1][i - 1]))")
        
for x in range(9):
    exec("string" + str(x) + " = 'hello'")

SyntaxError: invalid syntax (Temp/ipykernel_13692/3833544958.py, line 16)

In [ ]:
def r_star_generator(index):
    """
    r_star_generator computes r* given the column # of tree BUT must be done in numerical order, only starting from column/index 3 (D(1.5))
    :param index: the name of the column/period in tree being computed, D(T) = D(index/2)
    """
    m = GEKKO()
    j = 1 ### this value is meant to create new c# values below
    c1 = 1 ### initialize these c values
    c2 = 1
    ### tree['2'][0] = ru, tree['2'][1] = rd
    old_temp_list = [c1, c2]
                    
                                  
    for i in range(index - 1): ### range(index - 1) is the number of 'levels' deep we go with the c variables   
                                  
        if(i == index - 2):
            fraction = 0.5**(index - 1)
            for o in range(len(old_temp_list)):
                old_temp_list[o] = fraction
            break 
        else: 
            ### create/initialize the c's and update the j value
            temp_range2 = 2 ** (i + 2)
            new_temp_list = []
            for k in range(temp_range2): 
                exec("c" + str(j) + " = 1")
                new_temp_list.append(exec("c" + str(j)))
                j += 1
                          
                                     
            ### create list of ruu, rud, etc. in terms of r*
            r_star = m.Var()
            r_list = [r_star]
            for b in range(len(new_temp_list)):
                r_list.append(r_star * e**(-(2*b + 2) * vol_data[i + 1] * 0.70710678)) ### sqrt(0.5) = 0.70710678
                           
                                     
            ### set c's equal to lower levels (including the above c's)
            w = 0
            for o in range(len(old_temp_list)):
                old_temp_list[o] = new_temp_list[2*w]*(1 / (1 + r_list[2*o]/2)) + new_temp_list[2*w + 1]*(1 / (1 + r_list[2*o + 1]/2))
                w += 1
            old_temp_list = new_temp_list

    print(c1)
    m.Equation(term_structure[index - 1] == 1 / (1 + tree['1'][0]) * (c1 * (1 / (1 + tree['2'][0]/2)) + c2 * (1 / (1 + tree['2'][1]/2))))
    m.solve(disp = False)
    return(r_star[0])


In [ ]:
m = GEKKO()
y1 = m.Var()
y654 = m.Var()
y1 = 5
x = y1 * y654
i_list = [y1, y2]
print(x)

In [ ]:
r_star_generator(2)

In [127]:
def treeBuilder(index):
    """
    treeBuilder enters each r value for time 'index' into the correct column of tree dataframe
    :param index: the name of the column/period in tree being computed, T = index/2, D(T) 
    """
    fraction = 0.5**(index - 1)
    
    

In [128]:
print(tree)

     0         1         2    3    4    5    6    7    8    9  ...   21   22  \
0  NaN  0.056604   0.06327  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
1  NaN       NaN  0.054926  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2  NaN       NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
3  NaN       NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
4  NaN       NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
5  NaN       NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
6  NaN       NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
7  NaN       NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
8  NaN       NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
9  NaN       NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
10 NaN       NaN       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
11 NaN       NaN       NaN  NaN  NaN  Na